In [ ]:
# Extract results

In [ ]:
### Imports 
import os
import math
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.patches as patches
from matplotlib.collections import LineCollection
import time
import json

import pandas as pd

sys.path.append('../../')
from script.conversion.bison.coordinates import rd_to_utm
from mnms.graph.layers import PublicTransportLayer, MultiLayerGraph, OriginDestinationLayer, SharedVehicleLayer
from mnms.generation.roads import generate_pt_line_road, generate_one_zone
from mnms.generation.layers import generate_layer_from_roads
from mnms.vehicles.veh_type import Tram, Metro, Bus, Bike
from mnms.generation.zones import generate_one_zone
from mnms.mobility_service.public_transport import PublicTransportMobilityService
from mnms.mobility_service.vehicle_sharing import VehicleSharingMobilityService
from mnms.time import TimeTable, Dt, Time
from mnms.io.graph import load_graph, save_graph, load_odlayer, save_transit_links
from mnms.tools.render import draw_roads, draw_line, draw_odlayer, draw_path, draw_veh_activity
#from mnms.tools.geometry import points_in_polygon, get_bounding_box
from mnms.time import Time

In [ ]:
### Parameters

# Files and directories
f = open('params.json')
params = json.load(f)

current_dir = os.getcwd()
indir = current_dir + '/inputs/'
outdir = current_dir + '/outputs/'

#coord_csv_filepath = indir + 'KV1_GVB_2609_2/Csv/POINT.csv' # file with coordinates of the network
#amsterdam_json_filepath = indir + 'new_network.json' # mlgraph with the road network only
#amsterdam_json_filepath_pt_transit = indir + "network_pt_transit.json"
#transit_path = indir + "transit.json"

In [ ]:
### Load network

mmgraph_pt = load_graph(indir + params["fn_network"])
df_stations = pd.read_csv(indir+params['fn_emoped_st_init'])

#df_emoped1 = pd.read_csv(indir + 'init_pos_emoped.csv')
#df_emoped2 = pd.read_csv(indir + 'init_pos_emoped.csv')

### Load demand

df_dmd = pd.read_csv(indir + params['fn_demand'], sep=";")

### Load odlayer
odlayer = load_odlayer(indir + params["fn_odlayer"])
x_od = []
y_od = []
for i in odlayer.origins.items():
    x = i[1][0]
    y = i[1][1]
    x_od.append(x)
    y_od.append(y)

In [ ]:
## Load results

df_emoped1_notax = pd.read_csv(outdir+'emoped_600_notax/'+'emoped1_veh.csv', sep=";")
df_path_notax = pd.read_csv(outdir+'emoped_600_notax/'+'path.csv', sep=";")
df_users_notax = pd.read_csv(outdir+'emoped_600_notax/'+'users.csv', sep=";")

df_emoped1_notax300 = pd.read_csv(outdir+'emoped_300_notax/'+'emoped1_veh.csv', sep=";")
df_path_notax300 = pd.read_csv(outdir+'emoped_300_notax/'+'path.csv', sep=";")
df_users_notax300 = pd.read_csv(outdir+'emoped_300_notax/'+'users.csv', sep=";")

df_emoped1_tax1 = pd.read_csv(outdir+'emoped_600_tax1/'+'emoped1_veh.csv', sep=";")
df_path_tax1 = pd.read_csv(outdir+'emoped_600_tax1/'+'path.csv', sep=";")
df_users_tax1 = pd.read_csv(outdir+'emoped_600_tax1/'+'users.csv', sep=";")

df_emoped1_tax05 = pd.read_csv(outdir+'emoped_600_tax05/'+'emoped1_veh.csv', sep=";")
df_path_tax05 = pd.read_csv(outdir+'emoped_600_tax05/'+'path.csv', sep=";")
df_users_tax05 = pd.read_csv(outdir+'emoped_600_tax05/'+'users.csv', sep=";")

df_path_noemoped = pd.read_csv(outdir+'no_emoped/'+'path.csv', sep=";")
df_users_noemoped = pd.read_csv(outdir+'no_emoped/'+'users.csv', sep=";")

In [ ]:
# Look vehicle trajectory
"""veh_id = 130
df = df_emoped1[df_emoped1["ID"] == int(veh_id)]
list_pos_emoped = np.zeros((len(df['POSITION']),2))
for i, row in enumerate(df['POSITION']):
    pos = row.split(' ')
    #plt.text(float(pos[0]), float(pos[1]), row.TIME)
    list_pos_emoped[i,:] = [float(pos[0]), float(pos[1])]"""

In [ ]:
len(df_dmd)

In [ ]:
# Look O/D for users wihtout paths
"""list_nomatch = []
list_origins = []
list_destinations = []
for i, row in df_path_notax[:].iterrows():
    if pd.isna(row['PATH']):
        list_nomatch.append(row['ID'])
        user = df_dmd[df_dmd['ID']==row['ID']]
        list_origins.append([float(o) for o in user['ORIGIN'].iloc[0].split(' ')])
        list_destinations.append([float(o) for o in user['DESTINATION'].iloc[0].split(' ')])"""

In [ ]:
"""id_user = '1066750-1'
o_user =  [float(o) for o in df_dmd[df_dmd['ID']==id_user]['ORIGIN'].iloc[0].split(' ')]
d_user =  [float(o) for o in df_dmd[df_dmd['ID']==id_user]['DESTINATION'].iloc[0].split(' ')]

dist_o = [(o_user[0]-x)**2 + (o_user[1]-y)**2 for (x,y) in zip(x_od,y_od)]
dist_d = [(d_user[0]-x)**2 + (d_user[1]-y)**2 for (x,y) in zip(x_od,y_od)]
print(np.argmin(dist_o), np.argmin(dist_d))"""

In [ ]:
polygon = np.asarray([
    [628000, 5.81e6],
    [632500, 5.808e6],
    [634000, 5.8055e6],
    [634000, 5.803e6],
    [633000, 5.801e6],
    [630500, 5.7995e6],
    [629000, 5.8e6],
    [625500, 5.8e6],
    [625500, 5.803e6],
    [625500, 5.806e6]
])

In [ ]:
x_st = []
y_st = []
for n in df_stations['closest_node']:
    x_st.append(mmgraph_pt.roads.nodes[n].position[0])
    y_st.append(mmgraph_pt.roads.nodes[n].position[1])

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
draw_roads(ax, mmgraph_pt.roads, color='grey', linkwidth=0.1, nodesize=0, draw_stops=False, node_label=False)

# Params of the visualization
colors = {'BUS': 'green', 'METRO': 'red', 'TRAM': 'skyblue'}

for layer in mmgraph_pt.layers.values():
    if type(layer) == PublicTransportLayer:
        for name, line in layer.lines.items():
                draw_line(ax, mmgraph_pt, line, color=colors[name[:name.find('_')]], 
                          linkwidth=0.4, nodesize=1, line_label=None, label_size=1, alpha=1., stopmarkeredgewidth=0.1)

#plt.plot(x_od, y_od, 'dk', alpha=0.4)

plt.plot(x_st, y_st, '*b')

#plt.plot(list_pos_emoped[:,0], list_pos_emoped[:,1], ':.')

"""for o,d in zip(list_origins, list_destinations):
    plt.plot([o[0], d[0]], [o[1], d[1]], 'm:', alpha=0.2)
    plt.plot(d[0], d[1], 'dm', alpha=0.2)
    plt.plot(o[0], o[1], 'om', alpha=0.2)"""

#plt.plot([o_user[0], d_user[0]], [o_user[1], d_user[1]], 'o-')
x=list(polygon[:,0])
x.append(polygon[0,0])
y=list(polygon[:,1])
y.append(polygon[0,1])
#plt.plot(x, y, '+--k')

legend = [Line2D([0, 1], [0, 1], marker='.', markersize=12, markeredgecolor='black', color='skyblue', linewidth=5,
            label='Tram line'),
          Line2D([0, 1], [0, 1], marker='.', markersize=12, markeredgecolor='black', color='red', linewidth=5,
            label='Metro line'),
          Line2D([0, 1], [0, 1], marker='.', markersize=12, markeredgecolor='black', color='green', linewidth=5,
            label='Bus line'),
          Line2D([0, 1], [0, 1], marker='.', markersize=0, markeredgecolor='grey', markerfacecolor='grey', color='grey', linewidth=1,
            label='Roads'),
          Line2D([0, 1], [0, 1], marker='*', markersize=10, color='b', linewidth=0,
            label='Virtual station')]

legend = plt.legend(handles=legend)
#plt.savefig(params['figdir']+'network.pdf')

## Modes shares

In [ ]:
def emoped_in_modes(modes):
    return 'emoped1' in modes

def pt_in_modes(modes):
    return ('BUS' in modes) or ('TRAM' in modes) or ('METRO' in modes)

In [ ]:
emoped_only = 0
pt_only = 0
combined = 0
for modes in df_path_notax['SERVICES']:
    if not pd.isna(modes):
        if emoped_in_modes(modes) and pt_in_modes(modes):
            combined+=1
        if emoped_in_modes(modes) and (not pt_in_modes(modes)):
            emoped_only+=1
        if (not emoped_in_modes(modes)) and pt_in_modes(modes):
            pt_only+=1
print('_notax', emoped_only, pt_only, combined)
sum = emoped_only + pt_only + combined
print(emoped_only/sum, pt_only/sum, combined/sum)

emoped_only = 0
pt_only = 0
combined = 0
for modes in df_path_notax300['SERVICES']:
    if not pd.isna(modes):
        if emoped_in_modes(modes) and pt_in_modes(modes):
            combined+=1
        if emoped_in_modes(modes) and (not pt_in_modes(modes)):
            emoped_only+=1
        if (not emoped_in_modes(modes)) and pt_in_modes(modes):
            pt_only+=1
print('_notax300', emoped_only, pt_only, combined)
sum = emoped_only + pt_only + combined
print(emoped_only/sum, pt_only/sum, combined/sum)

emoped_only = 0
pt_only = 0
combined = 0
for modes in df_path_tax1['SERVICES']:
    if not pd.isna(modes):
        if emoped_in_modes(modes) and pt_in_modes(modes):
            combined+=1
        if emoped_in_modes(modes) and (not pt_in_modes(modes)):
            emoped_only+=1
        if (not emoped_in_modes(modes)) and pt_in_modes(modes):
            pt_only+=1
print('_tax1', emoped_only, pt_only, combined)
sum = emoped_only + pt_only + combined
print(emoped_only/sum, pt_only/sum, combined/sum)

emoped_only = 0
pt_only = 0
combined = 0
for modes in df_path_tax05['SERVICES']:
    if not pd.isna(modes):
        if emoped_in_modes(modes) and pt_in_modes(modes):
            combined+=1
        if emoped_in_modes(modes) and (not pt_in_modes(modes)):
            emoped_only+=1
        if (not emoped_in_modes(modes)) and pt_in_modes(modes):
            pt_only+=1
print('_tax05', emoped_only, pt_only, combined)
sum = emoped_only + pt_only + combined
print(emoped_only/sum, pt_only/sum, combined/sum)

emoped_only = 0
pt_only = 0
combined = 0
for modes in df_path_noemoped['SERVICES']:
    if not pd.isna(modes):
        if emoped_in_modes(modes) and pt_in_modes(modes):
            combined+=1
        if emoped_in_modes(modes) and (not pt_in_modes(modes)):
            emoped_only+=1
        if (not emoped_in_modes(modes)) and pt_in_modes(modes):
            pt_only+=1
print('_noemoped', emoped_only, pt_only, combined)
sum = emoped_only + pt_only + combined
print(emoped_only/sum, pt_only/sum, combined/sum)

In [ ]:
#df_path[df_path['EVENT']!='DEPARTURE']

In [ ]:
print(len(df_path_notax['SERVICES']))#, len(list_origins))

In [ ]:
# 489 3534 193

## TTT/TTD

In [ ]:
# Compute total travel time/distance

list_id = df_users_notax['ID'].unique()
TTT = 0
TTD = 0
for id in list_id:
    df = df_users_notax[df_users_notax['ID']==id]
    TTT += Time(df['TIME'].iloc[-1]).to_seconds() - Time(df['TIME'].iloc[0]).to_seconds()
    TTD += df['DISTANCE'].iloc[-1]
print('---notax')
print(TTT/3600, TTT/60/len(list_id))
print(TTD*1e-3, TTD*1e-3/len(list_id))

list_id = df_users_notax300['ID'].unique()
TTT = 0
TTD = 0
for id in list_id:
    df = df_users_notax300[df_users_notax300['ID']==id]
    TTT += Time(df['TIME'].iloc[-1]).to_seconds() - Time(df['TIME'].iloc[0]).to_seconds()
    TTD += df['DISTANCE'].iloc[-1]
print('---notax300')
print(TTT/3600, TTT/60/len(list_id))
print(TTD*1e-3, TTD*1e-3/len(list_id))

list_id = df_users_tax1['ID'].unique()
TTT = 0
TTD = 0
for id in list_id:
    df = df_users_tax1[df_users_tax1['ID']==id]
    TTT += Time(df['TIME'].iloc[-1]).to_seconds() - Time(df['TIME'].iloc[0]).to_seconds()
    TTD += df['DISTANCE'].iloc[-1]
print('---tax1')
print(TTT/3600, TTT/60/len(list_id))
print(TTD*1e-3, TTD*1e-3/len(list_id))

list_id = df_users_tax05['ID'].unique()
TTT = 0
TTD = 0
for id in list_id:
    df = df_users_tax05[df_users_tax05['ID']==id]
    TTT += Time(df['TIME'].iloc[-1]).to_seconds() - Time(df['TIME'].iloc[0]).to_seconds()
    TTD += df['DISTANCE'].iloc[-1]
print('---tax05')
print(TTT/3600, TTT/60/len(list_id))
print(TTD*1e-3, TTD*1e-3/len(list_id))

list_id = df_users_noemoped['ID'].unique()
TTT = 0
TTD = 0
for id in list_id:
    df = df_users_noemoped[df_users_noemoped['ID']==id]
    TTT += Time(df['TIME'].iloc[-1]).to_seconds() - Time(df['TIME'].iloc[0]).to_seconds()
    TTD += df['DISTANCE'].iloc[-1]
print('---noemoped')
print(TTT/3600, TTT/60/len(list_id))
print(TTD*1e-3, TTD*1e-3/len(list_id))

## Emoped usage

In [ ]:
nb_drive_notax = []
t_old = 0
n_loc = 0
for t, status in zip(df_emoped1_notax.TIME, df_emoped1_notax.STATE):
    if t!=t_old:
        t_old = t
        nb_drive_notax.append(n_loc)
        n_loc = 0
    if status=='SERVING':
        n_loc+=1

nb_drive_notax300 = []
t_old = 0
n_loc = 0
for t, status in zip(df_emoped1_notax300.TIME, df_emoped1_notax300.STATE):
    if t!=t_old:
        t_old = t
        nb_drive_notax300.append(n_loc)
        n_loc = 0
    if status=='SERVING':
        n_loc+=1

nb_drive_tax = []
t_old = 0
n_loc = 0
for t, status in zip(df_emoped1_tax1.TIME, df_emoped1_tax1.STATE):
    if t!=t_old:
        t_old = t
        nb_drive_tax.append(n_loc)
        n_loc = 0
    if status=='SERVING':
        n_loc+=1

In [ ]:
plt.plot(nb_drive_notax)
plt.plot(nb_drive_notax300)
plt.plot(nb_drive_tax)

In [ ]:
list_emoped = df_emoped1_notax['ID'].unique()
TTD = 0
for id in list_emoped:
    df = df_emoped1_notax[df_emoped1_notax['ID']==id]
    TTD += df['DISTANCE'].iloc[-1]
print('---notax')
print(TTD*1e-3)

list_emoped = df_emoped1_notax300['ID'].unique()
TTD = 0
for id in list_emoped:
    df = df_emoped1_notax300[df_emoped1_notax300['ID']==id]
    TTD += df['DISTANCE'].iloc[-1]
print('---notax300')
print(TTD*1e-3)

list_emoped = df_emoped1_tax1['ID'].unique()
TTD = 0
for id in list_emoped:
    df = df_emoped1_tax1[df_emoped1_tax1['ID']==id]
    TTD += df['DISTANCE'].iloc[-1]
print('---tax1')
print(TTD*1e-3)

list_emoped = df_emoped1_tax05['ID'].unique()
TTD = 0
for id in list_emoped:
    df = df_emoped1_tax05[df_emoped1_tax05['ID']==id]
    TTD += df['DISTANCE'].iloc[-1]
print('---tax05')
print(TTD*1e-3)